In [1]:
print("hello world")

hello world


In [2]:
!pip install opencv-python


In [3]:
import cv2
import numpy as np
import mediapipe as mp




In [6]:
import cv2
import mediapipe as mp

mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

with mp_hands.Hands(
    max_num_hands=10,  
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
) as hands:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Kamera görüntüsü alınamadı")
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)

        total_fingers = 0 

        if results.multi_hand_landmarks:
            for hand_landmarks, hand_handedness in zip(results.multi_hand_landmarks,
                                                       results.multi_handedness):

                hand_label = hand_handedness.classification[0].label  
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                landmarks = hand_landmarks.landmark
                fingers = []


                if hand_label == "Right":
                    if landmarks[4].x < landmarks[3].x:
                        fingers.append(1)
                    else:
                        fingers.append(0)
                else:  
                    if landmarks[4].x > landmarks[3].x:
                        fingers.append(1)
                    else:
                        fingers.append(0)


                tip_ids = [8, 12, 16, 20]
                for tip in tip_ids:
                    if landmarks[tip].y < landmarks[tip - 2].y:
                        fingers.append(1)
                    else:
                        fingers.append(0)

                finger_count = sum(fingers)
                total_fingers += finger_count


        cv2.rectangle(frame, (40, 60), (500, 110), (0, 0, 0), -1) 
        cv2.putText(frame, f"Total Fingers: {total_fingers}", (50, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)

        cv2.imshow("Kamera", frame)

        if cv2.waitKey(1) != -1:
            break

cap.release()
cv2.destroyAllWindows()
